# Knowledge Distillation on MNIST
Knowledge distillation is the process of transferring the higher performance of a more expensive model to a smaller one.  In this notebook, we will explore performing this process on MNIST.  To begin with, I have provided access to pre-trained model that is large, but performant.  The exact architecture is not relevant (although you can inspect this easily if you wish).  It is straightforward to load in pytorch with

In [12]:
import torch
device = 'cpu'

class Net(torch.nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.l1 = torch.nn.Linear(28**2,800)
        self.l2 = torch.nn.Linear(800,800)
        self.l3 = torch.nn.Linear(800,10)
        self.dropout2 = torch.nn.Dropout(0.5)
        self.dropout3 = torch.nn.Dropout(0.5)

    def forward(self, x):
        x = self.l1(x)
        x = torch.relu(x)
        x = self.dropout2(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.dropout3(x)
        x = self.l3(x)
        return x
    
big_model = torch.load('pretrained_model.pt').to(device)

First, let's establish the baseline performance of the big model on the MNIST test set.  Of course we'll need acces to the MNIST test set to do this.  At the same time, let's also get our transfer set, which in this case will be a $n=10$k subset of the full MNIST training set (using a subset is helpful for speeding up training of distilled models, and also helps showcase some of the improved performance due to model distillation).   

In [13]:
from torchvision import transforms, datasets
transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,)),
    ])

dataset_train = datasets.MNIST('./data', train=True, download=True, transform=transform)

dataset_test = datasets.MNIST('../data', train=False, download=True, transform=transform)

# This is a useful function that I didn't know about before
first_10k = list(range(0, 10000))
dataset_transfer = torch.utils.data.Subset(dataset_train, first_10k)

batch_size = 32
num_workers = 4
transfer_loader = torch.utils.data.DataLoader(dataset_transfer,batch_size=batch_size,num_workers=num_workers)
test_loader = torch.utils.data.DataLoader(dataset_test,batch_size=batch_size,num_workers=num_workers)

Here's a function that runs the big model in test mode and provides the number of correct examples

In [14]:
def test(model,test_loader):
    correct = 0
    counter = 0
    model.eval()
    with torch.no_grad():
        for data,target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            logits = model(data)
            pred = logits.argmax(dim=1,keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
            counter += batch_size
    return correct

test(big_model,test_loader)

9833

We find that the big model gets 167 examples wrong (not quite as good as the Hinton paper, but who cares). 

Now we would like to perform knowledge distillation by training a smaller model to approximate the larger model's performance on the transfer set.  First, let's build a smaller model.  You may use whatever architecture you choose, but I found that using two hidden layers, each with 200 units along with ReLU activations (and no regularization at all) worked fine.

In [15]:
class SmallNet(torch.nn.Module):
    def __init__(self):
        super(SmallNet, self).__init__()
        # Build a SmallNet
        self.l1 = torch.nn.Linear(28**2, 400)
        self.l2 = torch.nn.Linear(400, 400)
        self.l3 = torch.nn.Linear(400, 10)

    def forward(self, x):
        # Don't forget to put the right operations here too!
        x = self.l1(x)
        x = torch.relu(x)
        x = self.l2(x)
        x = torch.relu(x)
        x = self.l3(x)
        return x
    
small_model = SmallNet()
small_model.to(device)

SmallNet(
  (l1): Linear(in_features=784, out_features=400, bias=True)
  (l2): Linear(in_features=400, out_features=400, bias=True)
  (l3): Linear(in_features=400, out_features=10, bias=True)
)

**To establish a baseline performance level, train the small model on the transfer set**  

In [16]:
# I'm giving you this training function: you'll need to modify it below to do knowledge distillation
def train(model,train_loader,n_epochs):
    optimizer = torch.optim.Adam(model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(train_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            logits = model(data)
            L = loss_fn(logits,target)
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)

train(small_model,transfer_loader,50)

0 tensor(0.4145)
1 tensor(0.1704)
2 tensor(0.1022)
3 tensor(0.0847)
4 tensor(0.0544)
5 tensor(0.0446)
6 tensor(0.0378)
7 tensor(0.0522)
8 tensor(0.0350)
9 tensor(0.0252)
10 tensor(0.0351)
11 tensor(0.0320)
12 tensor(0.0213)
13 tensor(0.0246)
14 tensor(0.0199)
15 tensor(0.0273)
16 tensor(0.0257)
17 tensor(0.0190)
18 tensor(0.0092)
19 tensor(0.0134)
20 tensor(0.0165)
21 tensor(0.0319)
22 tensor(0.0219)
23 tensor(0.0328)
24 tensor(0.0156)
25 tensor(0.0097)
26 tensor(0.0107)
27 tensor(0.0143)
28 tensor(0.0150)
29 tensor(0.0102)
30 tensor(0.0082)
31 tensor(0.0214)
32 tensor(0.0191)
33 tensor(0.0160)
34 tensor(0.0214)
35 tensor(0.0190)
36 tensor(0.0194)
37 tensor(0.0036)
38 tensor(0.0095)
39 tensor(0.0100)
40 tensor(0.0002)
41 tensor(1.6237e-05)
42 tensor(1.2766e-05)
43 tensor(1.0679e-05)
44 tensor(9.1268e-06)
45 tensor(7.8916e-06)
46 tensor(6.8736e-06)
47 tensor(6.0027e-06)
48 tensor(5.2470e-06)
49 tensor(4.5937e-06)


**Evaluate the small model on the test set, and comment on its accuracy relative to the big model.**  As you might expect, the performance is relatively worse.  

In [17]:
num_correct = test(small_model, test_loader)
print('number wrong: ', 10000 - num_correct)

number wrong:  335


**The primary task of this notebook is now as follows: create a new training function similar to "train" above, but instead called "distill".**  "distill" should perform knowledge distillation as outlined in this week's paper.  It should accept a few additional arguments compared to train, namely the big model, the temperature hyperparameter, and a hyperparameter $\alpha$ that weights the relative magnitude of the soft target loss and the hard target loss.

In [23]:
distilled_model = SmallNet()
distilled_model.to(device)

# The body of this method is currently copied verbatim from the train method above: 
# you will need to modify it to utilize the big_model, temperature, and alpha values 
# to perform knowledge distillation
def distill(small_model,big_model,T,alpha,transfer_loader,n_epochs):
    optimizer = torch.optim.Adam(small_model.parameters(),1e-3)
    loss_fn = torch.nn.CrossEntropyLoss()
    small_model.train()
    for epoch in range(n_epochs):
        avg_l = 0.0
        counter = 0
        for batch_idx, (data, target) in enumerate(transfer_loader):
            data, target = data.to(device), target.to(device)
            data = data.reshape(data.shape[0],-1)
            optimizer.zero_grad()
            
            big_logits = big_model(data)
            softo = torch.nn.Softmax(dim=1)
            big_logits = softo(big_logits/T)
            small_logits = small_model(data)
            
            L_distill = loss_fn(small_logits, target)
            L_student = loss_fn(small_logits/T, big_logits)
            
            L = ((1 - alpha) * L_distill + alpha * L_student)
            
            L.backward()
            optimizer.step()
            with torch.no_grad():
                avg_l += L
                counter += 1
        print(epoch,avg_l/counter)

T = 10
alpha = .8
distill(distilled_model,big_model,T,alpha,transfer_loader,50)

0 tensor(1.1485)
1 tensor(0.8840)
2 tensor(0.8098)
3 tensor(0.7680)
4 tensor(0.7421)
5 tensor(0.7303)
6 tensor(0.7212)
7 tensor(0.7146)
8 tensor(0.7087)
9 tensor(0.7093)
10 tensor(0.7208)
11 tensor(0.7219)
12 tensor(0.7154)
13 tensor(0.7056)
14 tensor(0.7011)
15 tensor(0.6991)
16 tensor(0.6977)
17 tensor(0.6967)
18 tensor(0.6960)
19 tensor(0.7328)
20 tensor(0.7326)
21 tensor(0.7056)
22 tensor(0.6997)
23 tensor(0.6973)
24 tensor(0.6961)
25 tensor(0.6953)
26 tensor(0.6948)
27 tensor(0.6943)
28 tensor(0.6940)
29 tensor(0.6938)
30 tensor(0.7009)
31 tensor(0.7265)
32 tensor(0.7059)
33 tensor(0.6990)
34 tensor(0.6959)
35 tensor(0.6946)
36 tensor(0.6939)
37 tensor(0.6935)
38 tensor(0.6932)
39 tensor(0.6930)
40 tensor(0.6931)
41 tensor(0.6945)
42 tensor(0.7020)
43 tensor(0.7139)
44 tensor(0.7016)
45 tensor(0.6963)
46 tensor(0.6944)
47 tensor(0.6935)
48 tensor(0.6929)
49 tensor(0.6927)


**Finally, test your distilled model (on the test set) and describe how it performs relative to both big and small models.**

In [24]:
num_correct = test(distilled_model, test_loader)
print('number wrong: ', 10000 - num_correct)

number wrong:  259
